In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'../..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

hyspeclab_dir = os.path.join(project_dir, 'HySpecLab')
if hyspeclab_dir not in sys.path:
    sys.path.append(hyspeclab_dir)

ipdl_dir = os.path.join(project_dir, 'modules/IPDL')
if ipdl_dir not in sys.path:
    sys.path.append(ipdl_dir)    

from scipy.io import loadmat
from matplotlib import pyplot as plt
import numpy as np
import config

from HySpecLab.utils import HyperSpectralCalibration as calibration
from HySpecLab.utils import median

Este notebook es un ejemplo de clasificación usando el algoritmo de reducción de bandas (see ***HySpecLab.feature_selection.InterbandRedundancySelector***). El resultado final corresponde a la siguiente imagen donde se segmentan los distintos elementos de la imagen usando los datos de la imagen VNIR:

![Result](imgs/interband_polymer/interband_polymer_result.png)

# VNIR

In [ ]:
X = np.load(os.path.join(config.POLYMER_DATASET_DIR, 'Sensor_VNIR/C1_VNIR/processed/C1_VNIR.npy'))
tiny_X = X.reshape((X.shape[0]*X.shape[1],X.shape[2]))

# Band Selection
Due to computational problems, the sample is reduced to a single line for Band Selection based in Interband Redundancy.

In [ ]:
sub_X = X[200:201, 100:-100:2].squeeze() # Una linea reducida
fig, ax = plt.subplots(1,1,figsize=(20,16))
ax.imshow(sub_X[np.newaxis, :, 100])
ax.set_yticklabels([])
plt.show()

In [ ]:
from HySpecLab.transform import StandarizedTransform

sub_X_std = StandarizedTransform().fit_transform(sub_X)

In [ ]:
from HySpecLab.feature_selection import InterbandRedundancySelector

selector = InterbandRedundancySelector(threshold=16)
selector.fit(sub_X_std)

n_features = selector.mask_.sum().item()
features_selected = np.argwhere(selector.mask_ != 0).flatten()
print('Features selected: {}'.format(np.argwhere(selector.mask_ != 0).flatten()))

In [ ]:
tiny_X = selector.transform(tiny_X)
X = tiny_X.reshape(X.shape[0], X.shape[1], n_features)

## Normalize

In [ ]:
from HySpecLab.transform import Norm1Transform, StandarizedTransform

tiny_X_norm = Norm1Transform().fit_transform(tiny_X)
tiny_X_std = StandarizedTransform().fit_transform(tiny_X)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16,4))

samples_idx = np.arange(0, 1200, 400)

ax[0].set_title('Original')
ax[0].plot(tiny_X[samples_idx].T)
ax[1].set_title('1-Norm')
ax[1].plot(tiny_X_norm[samples_idx].T)
ax[2].set_title('Std Norm')
ax[2].plot(tiny_X_std[samples_idx].T)


plt.show()

# PCA Classification

In [ ]:
w, h, b = X.shape

In [ ]:
from sklearn.decomposition import PCA

n_components = 3

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(tiny_X.copy())
X_pca = X_pca.reshape(w, h, n_components)

X_pca_norm = pca.fit_transform(tiny_X_norm.copy())
X_pca_norm = X_pca_norm.reshape(w, h, n_components)

X_pca_std = pca.fit_transform(tiny_X_std.copy())
X_pca_std = X_pca_std.reshape(w, h, n_components)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=n_components, figsize=(16,8))

for i in range(n_components):
    ax[0, i].imshow(X_pca[:,:,i])
    ax[0, i].set_title('Component {}'.format(i))
    ax[0, i].set_xticklabels([])
    ax[0, i].set_yticklabels([])

ax[0, 0].set_ylabel('Original')

for i in range(n_components):
    ax[1, i].imshow(X_pca_norm[:,:,i])
    ax[1, i].set_xticklabels([])
    ax[1, i].set_yticklabels([])

ax[1, 0].set_ylabel('1-Norm')

for i in range(n_components):
    ax[2, i].imshow(X_pca_std[:,:,i])
    ax[2, i].set_xticklabels([])
    ax[2, i].set_yticklabels([])

ax[2, 0].set_ylabel('Std Norm')

plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16,4))

ax[0].set_title('Original')
ax[0].set_xlabel('Commponent 1')
ax[0].set_ylabel('Commponent 0')
ax[0].scatter(X_pca[:,:, 1].flatten()[::50], X_pca[:,:, 0].flatten()[::50])

ax[1].set_title('1-Norm')
ax[1].set_xlabel('Commponent 1')
ax[1].set_ylabel('Commponent 0')
ax[1].scatter(X_pca_norm[:,:, 1].flatten()[::50], X_pca_norm[:,:, 0].flatten()[::50])

ax[2].set_title('Std Norm')
ax[2].set_xlabel('Commponent 1')
ax[2].set_ylabel('Commponent 0')
ax[2].scatter(X_pca_std[:,:, 1].flatten()[::50], X_pca_std[:,:, 0].flatten()[::50])
plt.show()

# Visualization

In [ ]:
# data = X_pca
data = X_pca_norm
# data = X_pca_std

In [ ]:
fig, ax = plt.subplots(nrows=n_components, ncols=n_components, figsize=(16, 16))

for i in range(n_components):
    for j in range(n_components):
        ax[i,j].scatter(data[:,:, i].flatten()[::50], data[:,:, j].flatten()[::50])
        ax[i,j].set_xlabel('Commponent {}'.format(i))
        ax[i,j].set_ylabel('Commponent {}'.format(j))

plt.show()

In [ ]:
cmp_0 = data[:,:, 0].flatten().reshape(-1,1)
cmp_1 = data[:,:, 1].flatten().reshape(-1,1)

X = np.concatenate([cmp_0, cmp_1], axis=1)

In [ ]:
from sklearn.cluster import KMeans
n_clusters = 3
model = KMeans(n_clusters=n_clusters, random_state=0).fit(X)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=n_clusters+1, figsize=(16,4))

result = model.labels_.reshape(w, h)

ax[0].imshow(result)
ax[0].set_xticklabels([])
ax[0].set_yticklabels([])

for i in range(n_clusters):
    mask = np.zeros(result.shape)
    mask[np.where(result==i)] = 1
    ax[i+1].imshow(mask)
    ax[i+1].set_title('Label {}'.format(i))
    ax[i+1].set_xticklabels([])
    ax[i+1].set_yticklabels([])

plt.show()